# 合并词汇
- 爬取时间 2024.12.19
- 小写
- 合并
- 去重

In [4]:
import pandas as pd

# 读取三个文件
coinmarketca_df = pd.read_excel('coinmarketca.xlsx')
decripto_df = pd.read_excel('Decripto.xlsx')
ethereum_df = pd.read_excel('ethereum.xlsx')

# 计数每个文件的词汇数量（去除标题行）
coinmarketca_count = len(coinmarketca_df) - 1  # 去掉标题行
decripto_count = len(decripto_df) - 1
ethereum_count = len(ethereum_df) - 1

# 打印每个文件的词汇数量
print(f"Coinmarketca 文件词汇数量: {coinmarketca_count}")
print(f"Decripto 文件词汇数量: {decripto_count}")
print(f"Ethereum 文件词汇数量: {ethereum_count}")

# 合并所有词汇
all_terms = pd.concat([coinmarketca_df, decripto_df, ethereum_df], ignore_index=True)

# 将所有词汇转成小写
all_terms['Glossary Terms'] = all_terms['Glossary Terms'].str.lower()

# 去重
all_terms = all_terms.drop_duplicates()

# 将合并后的词汇保存为一个新文件
all_terms.to_excel('combined_glossary.xlsx', index=False)

# 打印合并后词汇的总数
print(f"合并后的词汇数量（小写去重后）: {len(all_terms)}")


Coinmarketca 文件词汇数量: 1259
Decripto 文件词汇数量: 254
Ethereum 文件词汇数量: 194
合并后的词汇数量（小写去重后）: 1511


# 人工整理出术语集和同义词集
1. 对包含括号的术语进行检查。如果括号内是同义词或缩写，则整合到同义词表中，如“accepting risk (acceptance)”或“banking secrecy act (bsa)”；如果括号内为补充说明，则删除括号及其内容，如“accretion (of a discount)”；如果括号为单位，则删除该词汇，如“MicroBitcoin (uBTC)”。
2. 对包含“/”的术语进行检查。如果是多个词义的词组，则分条整合到同义词表中，并在分条在术语表中列出，如“accumulation/distribution indicator”；如果是一个词义的多种表达方式，则整理到同义词表中。如果表示汇率，则删除该词汇，如“eth/btc”。
3. 对包含方括号的术语进行检查。经检查，方括号内的信息为XXXXX，直接删除内容，如“fud [2021]”。
4. 删除无效字符，如“đ”。

# 利用同义词集处理术语集

# 

In [8]:
import pandas as pd
# 读取人工整理术语表和同义词表（无标题行）
terms_df = pd.read_excel('人工整理术语表.xlsx', header=None)
synonyms_df = pd.read_excel('人工整理同义词表.xlsx', header=None)

# 打印读取的数据，检查是否正确加载
# print(terms_df.head())
# print(synonyms_df.head())


In [11]:
# 创建一个词汇到主词（第一列）的映射字典
synonym_map = {}

# 遍历同义词表的每一行
for _, row in synonyms_df.iterrows():
    main_term = row[0]  # 同义词表的第一列（主词）
    for synonym in row[1:]:  # 后续列为同义词
        if pd.notna(synonym):  # 忽略NaN值
            synonym_map[synonym.strip().lower()] = main_term.strip().lower()

# 打印部分映射关系，检查是否正确
print(dict(list(synonym_map.items())[5:10]))


{'ath': 'all-time-high', 'all-time high': 'all-time-high', 'atl': 'all-time-low', 'all-time low': 'all-time-low', 'ask me anything': 'ama'}


In [12]:
# 处理人工整理术语表中的词汇
terms_processed = []

for term in terms_df.iloc[:, 0]:  # 只处理第一列的词汇
    term = term.strip().lower()  # 转换为小写
    # 如果词汇在同义词映射中，替换为主词
    if term in synonym_map:
        terms_processed.append(synonym_map[term])
    else:
        terms_processed.append(term)

# 转换为pandas的Series并去重
terms_processed_unique = pd.Series(terms_processed).unique()

# 打印处理前和处理后的计数
print(f"处理前的词汇数：{len(terms_df)}")
print(f"处理后的去重词汇数：{len(terms_processed_unique)}")


处理前的词汇数：1513
处理后的去重词汇数：1435


In [13]:
# 打印同义词表的行数
print(f"同义词表的行数：{len(synonyms_df)}")

同义词表的行数：253


In [14]:
# 将处理后的词汇输出到一个新的Excel文件或直接打印
processed_df = pd.DataFrame(terms_processed_unique, columns=['Processed Terms'])
processed_df.to_excel('人工整理术语表_同义词处理后.xlsx', index=False)

# 打印处理后的前几行，确保处理正确
print(processed_df.head())

                 Processed Terms
0                 accepting risk
1                   advance line
2                            aes
3  algorithmic market operations
4                   algo-trading
